<a href="https://colab.research.google.com/github/Stefano97/AN2DL_Project2/blob/master/Preprocessing_attempt.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
%tensorflow_version 2.x
import tensorflow as tf

In [0]:
!git clone https://github.com/Stefano97/AN2DL_Project2

In [0]:
! pip uninstall kaggle

In [0]:
!pip install kaggle

In [0]:
from google.colab import files

files.upload()

In [0]:
!mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [0]:
!kaggle competitions download -c ann-and-dl-image-segmentation

In [0]:
!unzip ./*.zip